In [1]:
#notebook that calculates the vectors for a player
from awpy.parser import DemoParser
from awpy.analytics.map_control import extract_teams_metadata, calc_frame_map_control_values, calculate_round_map_control_metrics
from awpy.visualization.plot import plot_frame_map_control, plot_round_map_control, plot_map_control_metrics
from awpy.data import NAV_CSV
from awpy.analytics import map_control
from awpy.visualization import plot
import matplotlib.pyplot as plt
from collections import defaultdict

import os
import matplotlib
%matplotlib inline

import shutil

import pandas as pd
import numpy as np

def add_to_mapcontrol(j):
    #i want to know what tiles are affected, and by how much
    #j is the tile id sent over by tiles.iloc[j,1]
    #now, given the tile id, i calculate map control
    map_control_value=map_control._bfs(map_name, [j], area_threshold=1/20)
    #this returns a defaultdict(list) of values
    a=np.zeros(vectorlength)
    for i in tiles['areaId']:
        for j in map_control_value: #j is the KEY of the dictionary, aka the areaId
            if i==j: #i.e. the values match
                index=tiles[tiles['areaId']==i].index[0]-tiles.index[0] #get the index of the tile+convert it to a number we can index a by
                a[index]=a[index]+map_control_value[j]

    return a

demo_parser=DemoParser(demofile = "g2-vs-astralis-m2-inferno.dem", 
    demo_id = "g2_astralis_groupstage_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]

###DO NOT EXECUTE THIS CELL UNLESS YOU WANT TO OVERWRITE EVERYTHING!!!!!!
#this cell defines the different vectors we consider
#there are 6 types - full buy CT, half buy CT, full eco CT, full buy T, half buy T, full eco T
vectorlength=tiles.shape[0]

vector_prefixes = [
    'full_buy_ct_',
    'half_buy_ct_',
    'full_eco_ct_',
    'full_buy_t_',
    'half_buy_t_',
    'full_eco_t_'
]

# List of suffixes for different vector types
vector_suffixes = [
    'kill_vector',
    'killdistance_vector',
    'flash_vector',
    'enemyflashtime_vector',
    'smoke_vector',
    'molly_vector',
    'he_vector',
    'bombplant_vector',
    'mapcontrolpersecond_vector',
    'death_vector',
    'position_vector'
    # Add more suffixes as needed
]

#Loop through prefixes and suffixes to generate file names and create DataFrames
for prefix in vector_prefixes:
    for suffix in vector_suffixes:
        file_name = f'{prefix}{suffix}'
        df = pd.DataFrame(np.zeros(vectorlength))
        df.to_csv(f'{file_name}.csv', index=False)

#this cell defines the total number of full buy rounds, half buy rounds, and full eco rounds per side very very important, will be used to find averages later
#first, define the variables
number_of_full_buy_ct_rounds=0
number_of_half_buy_ct_rounds=0
number_of_full_eco_ct_rounds=0
number_of_full_buy_t_rounds=0
number_of_half_buy_t_rounds=0
number_of_full_eco_t_rounds=0

#next, save them as a pandas dataframe
number_of_full_buy_ct_rounds=pd.DataFrame(np.array([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.array([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.array([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.array([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.array([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.array([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

pro='huNter-'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "vitality-vs-g2-m1-inferno.dem", 
    demo_id = "vitality_g2_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='huNter-'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "g2-vs-faze-m1-inferno.dem", 
    demo_id = "g2_astralis_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='huNter-'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "g2-vs-astralis-sf-m2-inferno.dem", 
    demo_id = "g2_astralis_sf_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='huNter-'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')


sourcepath='.'
sourcefiles = os.listdir(sourcepath)
destinationpath = 'huNter-/inferno'
for file in sourcefiles:
    if file.endswith('.csv'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

demo_parser=DemoParser(demofile = "g2-vs-astralis-m2-inferno.dem", 
    demo_id = "g2_astralis_groupstage_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]

###DO NOT EXECUTE THIS CELL UNLESS YOU WANT TO OVERWRITE EVERYTHING!!!!!!
#this cell defines the different vectors we consider
#there are 6 types - full buy CT, half buy CT, full eco CT, full buy T, half buy T, full eco T
vectorlength=tiles.shape[0]

vector_prefixes = [
    'full_buy_ct_',
    'half_buy_ct_',
    'full_eco_ct_',
    'full_buy_t_',
    'half_buy_t_',
    'full_eco_t_'
]

# List of suffixes for different vector types
vector_suffixes = [
    'kill_vector',
    'killdistance_vector',
    'flash_vector',
    'enemyflashtime_vector',
    'smoke_vector',
    'molly_vector',
    'he_vector',
    'bombplant_vector',
    'mapcontrolpersecond_vector',
    'death_vector',
    'position_vector'
    # Add more suffixes as needed
]

#Loop through prefixes and suffixes to generate file names and create DataFrames
for prefix in vector_prefixes:
    for suffix in vector_suffixes:
        file_name = f'{prefix}{suffix}'
        df = pd.DataFrame(np.zeros(vectorlength))
        df.to_csv(f'{file_name}.csv', index=False)

#this cell defines the total number of full buy rounds, half buy rounds, and full eco rounds per side very very important, will be used to find averages later
#first, define the variables
number_of_full_buy_ct_rounds=0
number_of_half_buy_ct_rounds=0
number_of_full_eco_ct_rounds=0
number_of_full_buy_t_rounds=0
number_of_half_buy_t_rounds=0
number_of_full_eco_t_rounds=0

#next, save them as a pandas dataframe
number_of_full_buy_ct_rounds=pd.DataFrame(np.array([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.array([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.array([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.array([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.array([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.array([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

pro='jks'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "vitality-vs-g2-m1-inferno.dem", 
    demo_id = "vitality_g2_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='jks'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "g2-vs-faze-m1-inferno.dem", 
    demo_id = "g2_astralis_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='jks'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "g2-vs-astralis-sf-m2-inferno.dem", 
    demo_id = "g2_astralis_sf_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='HooXi'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')


sourcepath='.'
sourcefiles = os.listdir(sourcepath)
destinationpath = 'jks/inferno'
for file in sourcefiles:
    if file.endswith('.csv'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

demo_parser=DemoParser(demofile = "g2-vs-astralis-m2-inferno.dem", 
    demo_id = "g2_astralis_groupstage_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]

###DO NOT EXECUTE THIS CELL UNLESS YOU WANT TO OVERWRITE EVERYTHING!!!!!!
#this cell defines the different vectors we consider
#there are 6 types - full buy CT, half buy CT, full eco CT, full buy T, half buy T, full eco T
vectorlength=tiles.shape[0]

vector_prefixes = [
    'full_buy_ct_',
    'half_buy_ct_',
    'full_eco_ct_',
    'full_buy_t_',
    'half_buy_t_',
    'full_eco_t_'
]

# List of suffixes for different vector types
vector_suffixes = [
    'kill_vector',
    'killdistance_vector',
    'flash_vector',
    'enemyflashtime_vector',
    'smoke_vector',
    'molly_vector',
    'he_vector',
    'bombplant_vector',
    'mapcontrolpersecond_vector',
    'death_vector',
    'position_vector'
    # Add more suffixes as needed
]

#Loop through prefixes and suffixes to generate file names and create DataFrames
for prefix in vector_prefixes:
    for suffix in vector_suffixes:
        file_name = f'{prefix}{suffix}'
        df = pd.DataFrame(np.zeros(vectorlength))
        df.to_csv(f'{file_name}.csv', index=False)

#this cell defines the total number of full buy rounds, half buy rounds, and full eco rounds per side very very important, will be used to find averages later
#first, define the variables
number_of_full_buy_ct_rounds=0
number_of_half_buy_ct_rounds=0
number_of_full_eco_ct_rounds=0
number_of_full_buy_t_rounds=0
number_of_half_buy_t_rounds=0
number_of_full_eco_t_rounds=0

#next, save them as a pandas dataframe
number_of_full_buy_ct_rounds=pd.DataFrame(np.array([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.array([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.array([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.array([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.array([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.array([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

pro='m0NESY'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "vitality-vs-g2-m1-inferno.dem", 
    demo_id = "vitality_g2_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='m0NESY'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "g2-vs-faze-m1-inferno.dem", 
    demo_id = "g2_astralis_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='m0NESY'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "g2-vs-astralis-sf-m2-inferno.dem", 
    demo_id = "g2_astralis_sf_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='m0NESY'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')


sourcepath='.'
sourcefiles = os.listdir(sourcepath)
destinationpath = 'm0NESY/inferno'
for file in sourcefiles:
    if file.endswith('.csv'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

demo_parser=DemoParser(demofile = "g2-vs-astralis-m2-inferno.dem", 
    demo_id = "g2_astralis_groupstage_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]

###DO NOT EXECUTE THIS CELL UNLESS YOU WANT TO OVERWRITE EVERYTHING!!!!!!
#this cell defines the different vectors we consider
#there are 6 types - full buy CT, half buy CT, full eco CT, full buy T, half buy T, full eco T
vectorlength=tiles.shape[0]

vector_prefixes = [
    'full_buy_ct_',
    'half_buy_ct_',
    'full_eco_ct_',
    'full_buy_t_',
    'half_buy_t_',
    'full_eco_t_'
]

# List of suffixes for different vector types
vector_suffixes = [
    'kill_vector',
    'killdistance_vector',
    'flash_vector',
    'enemyflashtime_vector',
    'smoke_vector',
    'molly_vector',
    'he_vector',
    'bombplant_vector',
    'mapcontrolpersecond_vector',
    'death_vector',
    'position_vector'
    # Add more suffixes as needed
]

#Loop through prefixes and suffixes to generate file names and create DataFrames
for prefix in vector_prefixes:
    for suffix in vector_suffixes:
        file_name = f'{prefix}{suffix}'
        df = pd.DataFrame(np.zeros(vectorlength))
        df.to_csv(f'{file_name}.csv', index=False)

#this cell defines the total number of full buy rounds, half buy rounds, and full eco rounds per side very very important, will be used to find averages later
#first, define the variables
number_of_full_buy_ct_rounds=0
number_of_half_buy_ct_rounds=0
number_of_full_eco_ct_rounds=0
number_of_full_buy_t_rounds=0
number_of_half_buy_t_rounds=0
number_of_full_eco_t_rounds=0

#next, save them as a pandas dataframe
number_of_full_buy_ct_rounds=pd.DataFrame(np.array([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.array([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.array([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.array([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.array([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.array([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

pro='HooXi'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "vitality-vs-g2-m1-inferno.dem", 
    demo_id = "vitality_g2_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='jks'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "g2-vs-faze-m1-inferno.dem", 
    demo_id = "g2_astralis_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='HooXi'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "g2-vs-astralis-sf-m2-inferno.dem", 
    demo_id = "g2_astralis_sf_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='HooXi'
team_name='G2 Esports'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')


sourcepath='.'
sourcefiles = os.listdir(sourcepath)
destinationpath = 'HooXi/inferno'
for file in sourcefiles:
    if file.endswith('.csv'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

reached full buy ct
reached half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a tile in half buy ct
found a 

KeyboardInterrupt: 

In [ ]:
#notebook that calculates the vectors for a player
from awpy.parser import DemoParser
from awpy.analytics.map_control import extract_teams_metadata, calc_frame_map_control_values, calculate_round_map_control_metrics
from awpy.visualization.plot import plot_frame_map_control, plot_round_map_control, plot_map_control_metrics
from awpy.data import NAV_CSV
from awpy.analytics import map_control
from awpy.visualization import plot
import matplotlib.pyplot as plt
from collections import defaultdict

import os
import matplotlib
%matplotlib inline

import shutil

import pandas as pd
import numpy as np

def add_to_mapcontrol(j):
    #i want to know what tiles are affected, and by how much
    #j is the tile id sent over by tiles.iloc[j,1]
    #now, given the tile id, i calculate map control
    map_control_value=map_control._bfs(map_name, [j], area_threshold=1/20)
    #this returns a defaultdict(list) of values
    a=np.zeros(vectorlength)
    for i in tiles['areaId']:
        for j in map_control_value: #j is the KEY of the dictionary, aka the areaId
            if i==j: #i.e. the values match
                index=tiles[tiles['areaId']==i].index[0]-tiles.index[0] #get the index of the tile+convert it to a number we can index a by
                a[index]=a[index]+map_control_value[j]

    return a


demo_parser=DemoParser(demofile = "g2-vs-faze-m1-inferno.dem", 
    demo_id = "g2_faze_groupstage_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]

###DO NOT EXECUTE THIS CELL UNLESS YOU WANT TO OVERWRITE EVERYTHING!!!!!!
#this cell defines the different vectors we consider
#there are 6 types - full buy CT, half buy CT, full eco CT, full buy T, half buy T, full eco T
vectorlength=tiles.shape[0]

vector_prefixes = [
    'full_buy_ct_',
    'half_buy_ct_',
    'full_eco_ct_',
    'full_buy_t_',
    'half_buy_t_',
    'full_eco_t_'
]

# List of suffixes for different vector types
vector_suffixes = [
    'kill_vector',
    'killdistance_vector',
    'flash_vector',
    'enemyflashtime_vector',
    'smoke_vector',
    'molly_vector',
    'he_vector',
    'bombplant_vector',
    'mapcontrolpersecond_vector',
    'death_vector',
    'position_vector'
    # Add more suffixes as needed
]

#Loop through prefixes and suffixes to generate file names and create DataFrames
for prefix in vector_prefixes:
    for suffix in vector_suffixes:
        file_name = f'{prefix}{suffix}'
        df = pd.DataFrame(np.zeros(vectorlength))
        df.to_csv(f'{file_name}.csv', index=False)

#this cell defines the total number of full buy rounds, half buy rounds, and full eco rounds per side very very important, will be used to find averages later
#first, define the variables
number_of_full_buy_ct_rounds=0
number_of_half_buy_ct_rounds=0
number_of_full_eco_ct_rounds=0
number_of_full_buy_t_rounds=0
number_of_half_buy_t_rounds=0
number_of_full_eco_t_rounds=0

#next, save them as a pandas dataframe
number_of_full_buy_ct_rounds=pd.DataFrame(np.array([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.array([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.array([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.array([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.array([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.array([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

pro='karrigan'
team_name='FaZe Clan'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

sourcepath='.'
sourcefiles = os.listdir(sourcepath)
destinationpath = 'karrigan/inferno'
for file in sourcefiles:
    if file.endswith('.csv'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

demo_parser=DemoParser(demofile = "g2-vs-faze-m1-inferno.dem", 
    demo_id = "g2_faze_groupstage_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]

###DO NOT EXECUTE THIS CELL UNLESS YOU WANT TO OVERWRITE EVERYTHING!!!!!!
#this cell defines the different vectors we consider
#there are 6 types - full buy CT, half buy CT, full eco CT, full buy T, half buy T, full eco T
vectorlength=tiles.shape[0]

vector_prefixes = [
    'full_buy_ct_',
    'half_buy_ct_',
    'full_eco_ct_',
    'full_buy_t_',
    'half_buy_t_',
    'full_eco_t_'
]

# List of suffixes for different vector types
vector_suffixes = [
    'kill_vector',
    'killdistance_vector',
    'flash_vector',
    'enemyflashtime_vector',
    'smoke_vector',
    'molly_vector',
    'he_vector',
    'bombplant_vector',
    'mapcontrolpersecond_vector',
    'death_vector',
    'position_vector'
    # Add more suffixes as needed
]

#Loop through prefixes and suffixes to generate file names and create DataFrames
for prefix in vector_prefixes:
    for suffix in vector_suffixes:
        file_name = f'{prefix}{suffix}'
        df = pd.DataFrame(np.zeros(vectorlength))
        df.to_csv(f'{file_name}.csv', index=False)

#this cell defines the total number of full buy rounds, half buy rounds, and full eco rounds per side very very important, will be used to find averages later
#first, define the variables
number_of_full_buy_ct_rounds=0
number_of_half_buy_ct_rounds=0
number_of_full_eco_ct_rounds=0
number_of_full_buy_t_rounds=0
number_of_half_buy_t_rounds=0
number_of_full_eco_t_rounds=0

#next, save them as a pandas dataframe
number_of_full_buy_ct_rounds=pd.DataFrame(np.array([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.array([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.array([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.array([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.array([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.array([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

pro='rain'
team_name='FaZe Clan'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

sourcepath='.'
sourcefiles = os.listdir(sourcepath)
destinationpath = 'rain/inferno'
for file in sourcefiles:
    if file.endswith('.csv'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

demo_parser=DemoParser(demofile = "g2-vs-faze-m1-inferno.dem", 
    demo_id = "g2_faze_groupstage_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]

###DO NOT EXECUTE THIS CELL UNLESS YOU WANT TO OVERWRITE EVERYTHING!!!!!!
#this cell defines the different vectors we consider
#there are 6 types - full buy CT, half buy CT, full eco CT, full buy T, half buy T, full eco T
vectorlength=tiles.shape[0]

vector_prefixes = [
    'full_buy_ct_',
    'half_buy_ct_',
    'full_eco_ct_',
    'full_buy_t_',
    'half_buy_t_',
    'full_eco_t_'
]

# List of suffixes for different vector types
vector_suffixes = [
    'kill_vector',
    'killdistance_vector',
    'flash_vector',
    'enemyflashtime_vector',
    'smoke_vector',
    'molly_vector',
    'he_vector',
    'bombplant_vector',
    'mapcontrolpersecond_vector',
    'death_vector',
    'position_vector'
    # Add more suffixes as needed
]

#Loop through prefixes and suffixes to generate file names and create DataFrames
for prefix in vector_prefixes:
    for suffix in vector_suffixes:
        file_name = f'{prefix}{suffix}'
        df = pd.DataFrame(np.zeros(vectorlength))
        df.to_csv(f'{file_name}.csv', index=False)

#this cell defines the total number of full buy rounds, half buy rounds, and full eco rounds per side very very important, will be used to find averages later
#first, define the variables
number_of_full_buy_ct_rounds=0
number_of_half_buy_ct_rounds=0
number_of_full_eco_ct_rounds=0
number_of_full_buy_t_rounds=0
number_of_half_buy_t_rounds=0
number_of_full_eco_t_rounds=0

#next, save them as a pandas dataframe
number_of_full_buy_ct_rounds=pd.DataFrame(np.array([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.array([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.array([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.array([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.array([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.array([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

pro='Twistzz'
team_name='FaZe Clan'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

sourcepath='.'
sourcefiles = os.listdir(sourcepath)
destinationpath = 'Twistzz/inferno'
for file in sourcefiles:
    if file.endswith('.csv'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

demo_parser=DemoParser(demofile = "g2-vs-faze-m1-inferno.dem", 
    demo_id = "g2_faze_groupstage_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]

###DO NOT EXECUTE THIS CELL UNLESS YOU WANT TO OVERWRITE EVERYTHING!!!!!!
#this cell defines the different vectors we consider
#there are 6 types - full buy CT, half buy CT, full eco CT, full buy T, half buy T, full eco T
vectorlength=tiles.shape[0]

vector_prefixes = [
    'full_buy_ct_',
    'half_buy_ct_',
    'full_eco_ct_',
    'full_buy_t_',
    'half_buy_t_',
    'full_eco_t_'
]

# List of suffixes for different vector types
vector_suffixes = [
    'kill_vector',
    'killdistance_vector',
    'flash_vector',
    'enemyflashtime_vector',
    'smoke_vector',
    'molly_vector',
    'he_vector',
    'bombplant_vector',
    'mapcontrolpersecond_vector',
    'death_vector',
    'position_vector'
    # Add more suffixes as needed
]

#Loop through prefixes and suffixes to generate file names and create DataFrames
for prefix in vector_prefixes:
    for suffix in vector_suffixes:
        file_name = f'{prefix}{suffix}'
        df = pd.DataFrame(np.zeros(vectorlength))
        df.to_csv(f'{file_name}.csv', index=False)

#this cell defines the total number of full buy rounds, half buy rounds, and full eco rounds per side very very important, will be used to find averages later
#first, define the variables
number_of_full_buy_ct_rounds=0
number_of_half_buy_ct_rounds=0
number_of_full_eco_ct_rounds=0
number_of_full_buy_t_rounds=0
number_of_half_buy_t_rounds=0
number_of_full_eco_t_rounds=0

#next, save them as a pandas dataframe
number_of_full_buy_ct_rounds=pd.DataFrame(np.array([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.array([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.array([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.array([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.array([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.array([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

pro='ropz'
team_name='FaZe Clan'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

sourcepath='.'
sourcefiles = os.listdir(sourcepath)
destinationpath = 'ropz/inferno'
for file in sourcefiles:
    if file.endswith('.csv'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

demo_parser=DemoParser(demofile = "g2-vs-faze-m1-inferno.dem", 
    demo_id = "g2_faze_groupstage_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]

###DO NOT EXECUTE THIS CELL UNLESS YOU WANT TO OVERWRITE EVERYTHING!!!!!!
#this cell defines the different vectors we consider
#there are 6 types - full buy CT, half buy CT, full eco CT, full buy T, half buy T, full eco T
vectorlength=tiles.shape[0]

vector_prefixes = [
    'full_buy_ct_',
    'half_buy_ct_',
    'full_eco_ct_',
    'full_buy_t_',
    'half_buy_t_',
    'full_eco_t_'
]

# List of suffixes for different vector types
vector_suffixes = [
    'kill_vector',
    'killdistance_vector',
    'flash_vector',
    'enemyflashtime_vector',
    'smoke_vector',
    'molly_vector',
    'he_vector',
    'bombplant_vector',
    'mapcontrolpersecond_vector',
    'death_vector',
    'position_vector'
    # Add more suffixes as needed
]

#Loop through prefixes and suffixes to generate file names and create DataFrames
for prefix in vector_prefixes:
    for suffix in vector_suffixes:
        file_name = f'{prefix}{suffix}'
        df = pd.DataFrame(np.zeros(vectorlength))
        df.to_csv(f'{file_name}.csv', index=False)

#this cell defines the total number of full buy rounds, half buy rounds, and full eco rounds per side very very important, will be used to find averages later
#first, define the variables
number_of_full_buy_ct_rounds=0
number_of_half_buy_ct_rounds=0
number_of_full_eco_ct_rounds=0
number_of_full_buy_t_rounds=0
number_of_half_buy_t_rounds=0
number_of_full_eco_t_rounds=0

#next, save them as a pandas dataframe
number_of_full_buy_ct_rounds=pd.DataFrame(np.array([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.array([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.array([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.array([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.array([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.array([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

pro='broky'
team_name='FaZe Clan'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

sourcepath='.'
sourcefiles = os.listdir(sourcepath)
destinationpath = 'broky/inferno'
for file in sourcefiles:
    if file.endswith('.csv'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

In [ ]:
#notebook that calculates the vectors for a player
from awpy.parser import DemoParser
from awpy.analytics.map_control import extract_teams_metadata, calc_frame_map_control_values, calculate_round_map_control_metrics
from awpy.visualization.plot import plot_frame_map_control, plot_round_map_control, plot_map_control_metrics
from awpy.data import NAV_CSV
from awpy.analytics import map_control
from awpy.visualization import plot
import matplotlib.pyplot as plt
from collections import defaultdict

import os
import matplotlib
%matplotlib inline

import shutil

import pandas as pd
import numpy as np

def add_to_mapcontrol(j):
    #i want to know what tiles are affected, and by how much
    #j is the tile id sent over by tiles.iloc[j,1]
    #now, given the tile id, i calculate map control
    map_control_value=map_control._bfs(map_name, [j], area_threshold=1/20)
    #this returns a defaultdict(list) of values
    a=np.zeros(vectorlength)
    for i in tiles['areaId']:
        for j in map_control_value: #j is the KEY of the dictionary, aka the areaId
            if i==j: #i.e. the values match
                index=tiles[tiles['areaId']==i].index[0]-tiles.index[0] #get the index of the tile+convert it to a number we can index a by
                a[index]=a[index]+map_control_value[j]

    return a


demo_parser=DemoParser(demofile = "g2-vs-astralis-m2-inferno.dem", 
    demo_id = "g2_astralis_groupstage_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]

###DO NOT EXECUTE THIS CELL UNLESS YOU WANT TO OVERWRITE EVERYTHING!!!!!!
#this cell defines the different vectors we consider
#there are 6 types - full buy CT, half buy CT, full eco CT, full buy T, half buy T, full eco T
vectorlength=tiles.shape[0]

vector_prefixes = [
    'full_buy_ct_',
    'half_buy_ct_',
    'full_eco_ct_',
    'full_buy_t_',
    'half_buy_t_',
    'full_eco_t_'
]

# List of suffixes for different vector types
vector_suffixes = [
    'kill_vector',
    'killdistance_vector',
    'flash_vector',
    'enemyflashtime_vector',
    'smoke_vector',
    'molly_vector',
    'he_vector',
    'bombplant_vector',
    'mapcontrolpersecond_vector',
    'death_vector',
    'position_vector'
    # Add more suffixes as needed
]

#Loop through prefixes and suffixes to generate file names and create DataFrames
for prefix in vector_prefixes:
    for suffix in vector_suffixes:
        file_name = f'{prefix}{suffix}'
        df = pd.DataFrame(np.zeros(vectorlength))
        df.to_csv(f'{file_name}.csv', index=False)

#this cell defines the total number of full buy rounds, half buy rounds, and full eco rounds per side very very important, will be used to find averages later
#first, define the variables
number_of_full_buy_ct_rounds=0
number_of_half_buy_ct_rounds=0
number_of_full_eco_ct_rounds=0
number_of_full_buy_t_rounds=0
number_of_half_buy_t_rounds=0
number_of_full_eco_t_rounds=0

#next, save them as a pandas dataframe
number_of_full_buy_ct_rounds=pd.DataFrame(np.array([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.array([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.array([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.array([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.array([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.array([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

pro='device'
team_name='Astralis'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "heroic-vs-astralis-m2-inferno.dem", 
    demo_id = "heroic_astralis", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='device'
team_name='Astralis'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')


demo_parser=DemoParser(demofile = "g2-vs-astralis-sf-m2-inferno.dem", 
    demo_id = "g2_astralis_sf_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='device'
team_name='Astralis'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

sourcepath='.'
sourcefiles = os.listdir(sourcepath)
destinationpath = 'device/inferno'
for file in sourcefiles:
    if file.endswith('.csv'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

demo_parser=DemoParser(demofile = "g2-vs-astralis-m2-inferno.dem", 
    demo_id = "g2_astralis_groupstage_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]

###DO NOT EXECUTE THIS CELL UNLESS YOU WANT TO OVERWRITE EVERYTHING!!!!!!
#this cell defines the different vectors we consider
#there are 6 types - full buy CT, half buy CT, full eco CT, full buy T, half buy T, full eco T
vectorlength=tiles.shape[0]

vector_prefixes = [
    'full_buy_ct_',
    'half_buy_ct_',
    'full_eco_ct_',
    'full_buy_t_',
    'half_buy_t_',
    'full_eco_t_'
]

# List of suffixes for different vector types
vector_suffixes = [
    'kill_vector',
    'killdistance_vector',
    'flash_vector',
    'enemyflashtime_vector',
    'smoke_vector',
    'molly_vector',
    'he_vector',
    'bombplant_vector',
    'mapcontrolpersecond_vector',
    'death_vector',
    'position_vector'
    # Add more suffixes as needed
]

#Loop through prefixes and suffixes to generate file names and create DataFrames
for prefix in vector_prefixes:
    for suffix in vector_suffixes:
        file_name = f'{prefix}{suffix}'
        df = pd.DataFrame(np.zeros(vectorlength))
        df.to_csv(f'{file_name}.csv', index=False)

#this cell defines the total number of full buy rounds, half buy rounds, and full eco rounds per side very very important, will be used to find averages later
#first, define the variables
number_of_full_buy_ct_rounds=0
number_of_half_buy_ct_rounds=0
number_of_full_eco_ct_rounds=0
number_of_full_buy_t_rounds=0
number_of_half_buy_t_rounds=0
number_of_full_eco_t_rounds=0

#next, save them as a pandas dataframe
number_of_full_buy_ct_rounds=pd.DataFrame(np.array([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.array([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.array([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.array([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.array([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.array([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

pro='Staehr'
team_name='Astralis'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "heroic-vs-astralis-m2-inferno.dem", 
    demo_id = "heroic_astralis", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='Staehr'
team_name='Astralis'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')


demo_parser=DemoParser(demofile = "g2-vs-astralis-sf-m2-inferno.dem", 
    demo_id = "g2_astralis_sf_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='Staehr'
team_name='Astralis'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

sourcepath='.'
sourcefiles = os.listdir(sourcepath)
destinationpath = 'Staehr/inferno'
for file in sourcefiles:
    if file.endswith('.csv'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

demo_parser=DemoParser(demofile = "g2-vs-astralis-m2-inferno.dem", 
    demo_id = "g2_astralis_groupstage_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]

###DO NOT EXECUTE THIS CELL UNLESS YOU WANT TO OVERWRITE EVERYTHING!!!!!!
#this cell defines the different vectors we consider
#there are 6 types - full buy CT, half buy CT, full eco CT, full buy T, half buy T, full eco T
vectorlength=tiles.shape[0]

vector_prefixes = [
    'full_buy_ct_',
    'half_buy_ct_',
    'full_eco_ct_',
    'full_buy_t_',
    'half_buy_t_',
    'full_eco_t_'
]

# List of suffixes for different vector types
vector_suffixes = [
    'kill_vector',
    'killdistance_vector',
    'flash_vector',
    'enemyflashtime_vector',
    'smoke_vector',
    'molly_vector',
    'he_vector',
    'bombplant_vector',
    'mapcontrolpersecond_vector',
    'death_vector',
    'position_vector'
    # Add more suffixes as needed
]

#Loop through prefixes and suffixes to generate file names and create DataFrames
for prefix in vector_prefixes:
    for suffix in vector_suffixes:
        file_name = f'{prefix}{suffix}'
        df = pd.DataFrame(np.zeros(vectorlength))
        df.to_csv(f'{file_name}.csv', index=False)

#this cell defines the total number of full buy rounds, half buy rounds, and full eco rounds per side very very important, will be used to find averages later
#first, define the variables
number_of_full_buy_ct_rounds=0
number_of_half_buy_ct_rounds=0
number_of_full_eco_ct_rounds=0
number_of_full_buy_t_rounds=0
number_of_half_buy_t_rounds=0
number_of_full_eco_t_rounds=0

#next, save them as a pandas dataframe
number_of_full_buy_ct_rounds=pd.DataFrame(np.array([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.array([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.array([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.array([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.array([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.array([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

pro='blameF'
team_name='Astralis'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "heroic-vs-astralis-m2-inferno.dem", 
    demo_id = "heroic_astralis", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='blameF'
team_name='Astralis'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')


demo_parser=DemoParser(demofile = "g2-vs-astralis-sf-m2-inferno.dem", 
    demo_id = "g2_astralis_sf_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='blameF'
team_name='Astralis'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

sourcepath='.'
sourcefiles = os.listdir(sourcepath)
destinationpath = 'blameF/inferno'
for file in sourcefiles:
    if file.endswith('.csv'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

demo_parser=DemoParser(demofile = "g2-vs-astralis-m2-inferno.dem", 
    demo_id = "g2_astralis_groupstage_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]

###DO NOT EXECUTE THIS CELL UNLESS YOU WANT TO OVERWRITE EVERYTHING!!!!!!
#this cell defines the different vectors we consider
#there are 6 types - full buy CT, half buy CT, full eco CT, full buy T, half buy T, full eco T
vectorlength=tiles.shape[0]

vector_prefixes = [
    'full_buy_ct_',
    'half_buy_ct_',
    'full_eco_ct_',
    'full_buy_t_',
    'half_buy_t_',
    'full_eco_t_'
]

# List of suffixes for different vector types
vector_suffixes = [
    'kill_vector',
    'killdistance_vector',
    'flash_vector',
    'enemyflashtime_vector',
    'smoke_vector',
    'molly_vector',
    'he_vector',
    'bombplant_vector',
    'mapcontrolpersecond_vector',
    'death_vector',
    'position_vector'
    # Add more suffixes as needed
]

#Loop through prefixes and suffixes to generate file names and create DataFrames
for prefix in vector_prefixes:
    for suffix in vector_suffixes:
        file_name = f'{prefix}{suffix}'
        df = pd.DataFrame(np.zeros(vectorlength))
        df.to_csv(f'{file_name}.csv', index=False)

#this cell defines the total number of full buy rounds, half buy rounds, and full eco rounds per side very very important, will be used to find averages later
#first, define the variables
number_of_full_buy_ct_rounds=0
number_of_half_buy_ct_rounds=0
number_of_full_eco_ct_rounds=0
number_of_full_buy_t_rounds=0
number_of_half_buy_t_rounds=0
number_of_full_eco_t_rounds=0

#next, save them as a pandas dataframe
number_of_full_buy_ct_rounds=pd.DataFrame(np.array([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.array([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.array([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.array([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.array([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.array([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

pro='b0RUP'
team_name='Astralis'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "heroic-vs-astralis-m2-inferno.dem", 
    demo_id = "heroic_astralis", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='b0RUP'
team_name='Astralis'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')


demo_parser=DemoParser(demofile = "g2-vs-astralis-sf-m2-inferno.dem", 
    demo_id = "g2_astralis_sf_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='b0RUP'
team_name='Astralis'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

sourcepath='.'
sourcefiles = os.listdir(sourcepath)
destinationpath = 'b0RUP/inferno'
for file in sourcefiles:
    if file.endswith('.csv'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

demo_parser=DemoParser(demofile = "g2-vs-astralis-m2-inferno.dem", 
    demo_id = "g2_astralis_groupstage_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]

###DO NOT EXECUTE THIS CELL UNLESS YOU WANT TO OVERWRITE EVERYTHING!!!!!!
#this cell defines the different vectors we consider
#there are 6 types - full buy CT, half buy CT, full eco CT, full buy T, half buy T, full eco T
vectorlength=tiles.shape[0]

vector_prefixes = [
    'full_buy_ct_',
    'half_buy_ct_',
    'full_eco_ct_',
    'full_buy_t_',
    'half_buy_t_',
    'full_eco_t_'
]

# List of suffixes for different vector types
vector_suffixes = [
    'kill_vector',
    'killdistance_vector',
    'flash_vector',
    'enemyflashtime_vector',
    'smoke_vector',
    'molly_vector',
    'he_vector',
    'bombplant_vector',
    'mapcontrolpersecond_vector',
    'death_vector',
    'position_vector'
    # Add more suffixes as needed
]

#Loop through prefixes and suffixes to generate file names and create DataFrames
for prefix in vector_prefixes:
    for suffix in vector_suffixes:
        file_name = f'{prefix}{suffix}'
        df = pd.DataFrame(np.zeros(vectorlength))
        df.to_csv(f'{file_name}.csv', index=False)

#this cell defines the total number of full buy rounds, half buy rounds, and full eco rounds per side very very important, will be used to find averages later
#first, define the variables
number_of_full_buy_ct_rounds=0
number_of_half_buy_ct_rounds=0
number_of_full_eco_ct_rounds=0
number_of_full_buy_t_rounds=0
number_of_half_buy_t_rounds=0
number_of_full_eco_t_rounds=0

#next, save them as a pandas dataframe
number_of_full_buy_ct_rounds=pd.DataFrame(np.array([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.array([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.array([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.array([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.array([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.array([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

pro='Buzz'
team_name='Astralis'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

demo_parser=DemoParser(demofile = "heroic-vs-astralis-m2-inferno.dem", 
    demo_id = "heroic_astralis", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='Buzz'
team_name='Astralis'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')


demo_parser=DemoParser(demofile = "g2-vs-astralis-sf-m2-inferno.dem", 
    demo_id = "g2_astralis_sf_inferno", 
    parse_rate=128, 
    trade_time=3, 
    buy_style="hltv",
)

data=demo_parser.parse(clean=True)
tiles=NAV_CSV[NAV_CSV["mapName"] == data["mapName"]]


pro='Buzz'
team_name='Astralis'
map_name='de_inferno'

full_buy_ct_rounds=[]
half_buy_ct_rounds=[]
full_eco_ct_rounds=[]
full_buy_t_rounds=[]
half_buy_t_rounds=[]
full_eco_t_rounds=[]

for i in data['gameRounds']:
    if i['ctTeam']==team_name:
        if i['ctBuyType']=='Full Buy':
            full_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Semi Buy':
            half_buy_ct_rounds.append(i)
        elif i['ctBuyType']=='Full Eco':
            full_eco_ct_rounds.append(i)
            
    elif i['tTeam']==team_name:
        if i['tBuyType']=='Full Buy':
            full_buy_t_rounds.append(i)
        if i['tBuyType']=='Semi Buy':
            half_buy_t_rounds.append(i)
        elif i['tBuyType']=='Full Eco':
            full_eco_t_rounds.append(i)

number_of_full_buy_ct_rounds=pd.read_csv('number_of_full_buy_ct_rounds.csv')
number_of_half_buy_ct_rounds=pd.read_csv('number_of_half_buy_ct_rounds.csv')
number_of_full_eco_ct_rounds=pd.read_csv('number_of_full_eco_ct_rounds.csv')
number_of_full_buy_t_rounds=pd.read_csv('number_of_full_buy_t_rounds.csv')
number_of_half_buy_t_rounds=pd.read_csv('number_of_half_buy_t_rounds.csv')
number_of_full_eco_t_rounds=pd.read_csv('number_of_full_eco_t_rounds.csv')

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

number_of_full_buy_ct_rounds=number_of_full_buy_ct_rounds.iloc[0]+len(full_buy_ct_temp)
number_of_half_buy_ct_rounds=number_of_half_buy_ct_rounds.iloc[0]+len(half_buy_ct_temp)
number_of_full_eco_ct_rounds=number_of_full_eco_ct_rounds.iloc[0]+len(full_eco_ct_temp)
number_of_full_buy_t_rounds=number_of_full_buy_t_rounds.iloc[0]+len(full_buy_t_temp)
number_of_half_buy_t_rounds=number_of_half_buy_t_rounds.iloc[0]+len(half_buy_t_temp)
number_of_full_eco_t_rounds=number_of_full_eco_t_rounds.iloc[0]+len(full_eco_t_temp)

#save with updated values

number_of_full_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_full_buy_ct_rounds]))
number_of_full_buy_ct_rounds.to_csv('number_of_full_buy_ct_rounds.csv',index=False)

number_of_half_buy_ct_rounds=pd.DataFrame(np.asarray([number_of_half_buy_ct_rounds]))
number_of_half_buy_ct_rounds.to_csv('number_of_half_buy_ct_rounds.csv',index=False)

number_of_full_eco_ct_rounds=pd.DataFrame(np.asarray([number_of_full_eco_ct_rounds]))
number_of_full_eco_ct_rounds.to_csv('number_of_full_eco_ct_rounds.csv',index=False)

number_of_full_buy_t_rounds=pd.DataFrame(np.asarray([number_of_full_buy_t_rounds]))
number_of_full_buy_t_rounds.to_csv('number_of_full_buy_t_rounds.csv',index=False)

number_of_half_buy_t_rounds=pd.DataFrame(np.asarray([number_of_half_buy_t_rounds]))
number_of_half_buy_t_rounds.to_csv('number_of_half_buy_t_rounds.csv',index=False)

number_of_full_eco_t_rounds=pd.DataFrame(np.asarray([number_of_full_eco_t_rounds]))
number_of_full_eco_t_rounds.to_csv('number_of_full_eco_t_rounds.csv',index=False)

#now the cells to feed data into the CSV
#this cell does it for kills and killdistance

#full buy ct
full_buy_ct_kill_vector=pd.read_csv('full_buy_ct_kill_vector.csv')
full_buy_ct_killdistance_vector=pd.read_csv('full_buy_ct_killdistance_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z (as Z also increases)
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_kill_vector.iloc[i]=full_buy_ct_kill_vector.iloc[i]+1
            full_buy_ct_killdistance_vector.iloc[i]=full_buy_ct_killdistance_vector.iloc[i]+j['distance']

#half buy ct
half_buy_ct_kill_vector=pd.read_csv('half_buy_ct_kill_vector.csv')
half_buy_ct_killdistance_vector=pd.read_csv('half_buy_ct_killdistance_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_kill_vector.iloc[i]=half_buy_ct_kill_vector.iloc[i]+1
            half_buy_ct_killdistance_vector.iloc[i]=half_buy_ct_killdistance_vector.iloc[i]+j['distance']

#full eco ct
full_eco_ct_kill_vector=pd.read_csv('full_eco_ct_kill_vector.csv')
full_eco_ct_killdistance_vector=pd.read_csv('full_eco_ct_killdistance_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_kill_vector.iloc[i]=full_eco_ct_kill_vector.iloc[i]+1
            full_eco_ct_killdistance_vector.iloc[i]=full_eco_ct_killdistance_vector.iloc[i]+j['distance']

#full buy t
full_buy_t_kill_vector=pd.read_csv('full_buy_t_kill_vector.csv')
full_buy_t_killdistance_vector=pd.read_csv('full_buy_t_killdistance_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_kill_vector.iloc[i]=full_buy_t_kill_vector.iloc[i]+1
            full_buy_t_killdistance_vector.iloc[i]=full_buy_t_killdistance_vector.iloc[i]+j['distance']

#half buy t
half_buy_t_kill_vector=pd.read_csv('half_buy_t_kill_vector.csv')
half_buy_t_killdistance_vector=pd.read_csv('half_buy_t_killdistance_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_kill_vector.iloc[i]=half_buy_t_kill_vector.iloc[i]+1
            half_buy_t_killdistance_vector.iloc[i]=half_buy_t_killdistance_vector.iloc[i]+j['distance']

#full eco t
full_eco_t_kill_vector=pd.read_csv('full_eco_t_kill_vector.csv')
full_eco_t_killdistance_vector=pd.read_csv('full_eco_t_killdistance_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['attackerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['attackerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['attackerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['attackerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['attackerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['attackerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_kill_vector.iloc[i]=full_eco_t_kill_vector.iloc[i]+1
            full_eco_t_killdistance_vector.iloc[i]=full_eco_t_killdistance_vector.iloc[i]+j['distance']

full_buy_ct_kill_vector.to_csv('full_buy_ct_kill_vector.csv',index=False)
half_buy_ct_kill_vector.to_csv('half_buy_ct_kill_vector.csv',index=False)
full_eco_ct_kill_vector.to_csv('full_eco_ct_kill_vector.csv',index=False)
full_buy_t_kill_vector.to_csv('full_buy_t_kill_vector.csv',index=False)
half_buy_t_kill_vector.to_csv('half_buy_t_kill_vector.csv',index=False)
full_eco_t_kill_vector.to_csv('full_eco_t_kill_vector.csv',index=False)
full_buy_ct_killdistance_vector.to_csv('full_buy_ct_killdistance_vector.csv',index=False)
half_buy_ct_killdistance_vector.to_csv('half_buy_ct_killdistance_vector.csv',index=False)
full_eco_ct_killdistance_vector.to_csv('full_eco_ct_killdistance_vector.csv',index=False)
full_buy_t_killdistance_vector.to_csv('full_buy_t_killdistance_vector.csv',index=False)
half_buy_t_killdistance_vector.to_csv('half_buy_t_killdistance_vector.csv',index=False)
full_eco_t_killdistance_vector.to_csv('full_eco_t_killdistance_vector.csv',index=False)

del full_buy_ct_kill_vector,half_buy_ct_kill_vector,full_eco_ct_kill_vector,full_buy_t_kill_vector,half_buy_t_kill_vector,full_eco_t_kill_vector
del full_buy_ct_killdistance_vector,half_buy_ct_killdistance_vector,full_eco_ct_killdistance_vector,full_buy_t_killdistance_vector,half_buy_t_killdistance_vector,full_eco_t_killdistance_vector

#this section gets the number of smoke grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Smoke Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for smokes

#full buy ct
full_buy_ct_smoke_vector=pd.read_csv('full_buy_ct_smoke_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_smoke_vector.iloc[i]=full_buy_ct_smoke_vector.iloc[i]+1

#half buy ct
half_buy_ct_smoke_vector=pd.read_csv('half_buy_ct_smoke_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_smoke_vector.iloc[i]=half_buy_ct_smoke_vector.iloc[i]+1

#full eco ct
full_eco_ct_smoke_vector=pd.read_csv('full_eco_ct_smoke_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_smoke_vector.iloc[i]=full_eco_ct_smoke_vector.iloc[i]+1

#full buy t
full_buy_t_smoke_vector=pd.read_csv('full_buy_t_smoke_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_smoke_vector.iloc[i]=full_buy_t_smoke_vector.iloc[i]+1

#half buy t
half_buy_t_smoke_vector=pd.read_csv('half_buy_t_smoke_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_smoke_vector.iloc[i]=half_buy_t_smoke_vector.iloc[i]+1

#full eco t
full_eco_t_smoke_vector=pd.read_csv('full_eco_t_smoke_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_smoke_vector.iloc[i]=full_eco_t_smoke_vector.iloc[i]+1

full_buy_ct_smoke_vector.to_csv('full_buy_ct_smoke_vector.csv',index=False)
half_buy_ct_smoke_vector.to_csv('half_buy_ct_smoke_vector.csv',index=False)
full_eco_ct_smoke_vector.to_csv('full_eco_ct_smoke_vector.csv',index=False)
full_buy_t_smoke_vector.to_csv('full_buy_t_smoke_vector.csv',index=False)
half_buy_t_smoke_vector.to_csv('half_buy_t_smoke_vector.csv',index=False)
full_eco_t_smoke_vector.to_csv('full_eco_t_smoke_vector.csv',index=False)

del full_buy_ct_smoke_vector,half_buy_ct_smoke_vector,full_eco_ct_smoke_vector,full_buy_t_smoke_vector,half_buy_t_smoke_vector,full_eco_t_smoke_vector

#this section gets the number of flash grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='Flashbang':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flashing in csv


#full buy ct
full_buy_ct_flash_vector=pd.read_csv('full_buy_ct_flash_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_flash_vector.iloc[i]=full_buy_ct_flash_vector.iloc[i]+1

#half buy ct
half_buy_ct_flash_vector=pd.read_csv('half_buy_ct_flash_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_flash_vector.iloc[i]=half_buy_ct_flash_vector.iloc[i]+1

#full eco ct
full_eco_ct_flash_vector=pd.read_csv('full_eco_ct_flash_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_flash_vector.iloc[i]=full_eco_ct_flash_vector.iloc[i]+1

#full buy t
full_buy_t_flash_vector=pd.read_csv('full_buy_t_flash_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_flash_vector.iloc[i]=full_buy_t_flash_vector.iloc[i]+1

#half buy t
half_buy_t_flash_vector=pd.read_csv('half_buy_t_flash_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_flash_vector.iloc[i]=half_buy_t_flash_vector.iloc[i]+1

#full eco t
full_eco_t_flash_vector=pd.read_csv('full_eco_t_flash_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_flash_vector.iloc[i]=full_eco_t_flash_vector.iloc[i]+1

full_buy_ct_flash_vector.to_csv('full_buy_ct_flash_vector.csv',index=False)
half_buy_ct_flash_vector.to_csv('half_buy_ct_flash_vector.csv',index=False)
full_eco_ct_flash_vector.to_csv('full_eco_ct_flash_vector.csv',index=False)
full_buy_t_flash_vector.to_csv('full_buy_t_flash_vector.csv',index=False)
half_buy_t_flash_vector.to_csv('half_buy_t_flash_vector.csv',index=False)
full_eco_t_flash_vector.to_csv('full_eco_t_flash_vector.csv',index=False)

del full_buy_ct_flash_vector,half_buy_ct_flash_vector,full_eco_ct_flash_vector,full_buy_t_flash_vector,half_buy_t_flash_vector,full_eco_t_flash_vector

#molotovs - reading

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and (i['grenadeType']=='Incendiary Grenade' or i['grenadeType']=='Molotov'):
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#mollying in csv

#full buy ct
full_buy_ct_molly_vector=pd.read_csv('full_buy_ct_molly_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_molly_vector.iloc[i]=full_buy_ct_molly_vector.iloc[i]+1

#half buy ct
half_buy_ct_molly_vector=pd.read_csv('half_buy_ct_molly_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_molly_vector.iloc[i]=half_buy_ct_molly_vector.iloc[i]+1

#full eco ct
full_eco_ct_molly_vector=pd.read_csv('full_eco_ct_molly_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_molly_vector.iloc[i]=full_eco_ct_molly_vector.iloc[i]+1

#full buy t
full_buy_t_molly_vector=pd.read_csv('full_buy_t_molly_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7]): #to the UP of SOUTH Y
            full_buy_t_molly_vector.iloc[i]=full_buy_t_molly_vector.iloc[i]+1

#half buy t
half_buy_t_molly_vector=pd.read_csv('half_buy_t_molly_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_molly_vector.iloc[i]=half_buy_t_molly_vector.iloc[i]+1

#full eco t
full_eco_t_molly_vector=pd.read_csv('full_eco_t_molly_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_molly_vector.iloc[i]=full_eco_t_molly_vector.iloc[i]+1

full_buy_ct_molly_vector.to_csv('full_buy_ct_molly_vector.csv',index=False)
half_buy_ct_molly_vector.to_csv('half_buy_ct_molly_vector.csv',index=False)
full_eco_ct_molly_vector.to_csv('full_eco_ct_molly_vector.csv',index=False)
full_buy_t_molly_vector.to_csv('full_buy_t_molly_vector.csv',index=False)
half_buy_t_molly_vector.to_csv('half_buy_t_molly_vector.csv',index=False)
full_eco_t_molly_vector.to_csv('full_eco_t_molly_vector.csv',index=False)

del full_buy_ct_molly_vector,half_buy_ct_molly_vector,full_eco_ct_molly_vector,full_buy_t_molly_vector,half_buy_t_molly_vector,full_eco_t_molly_vector

#this section gets the number of HE grenades for a certain type of round

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['grenades'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['grenades'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['grenades'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['grenades'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['grenades'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['grenades'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['throwerName']==pro and i['grenadeType']=='HE Grenade':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#HE nades in csv

#full buy ct
full_buy_ct_he_vector=pd.read_csv('full_buy_ct_he_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_he_vector.iloc[i]=full_buy_ct_he_vector.iloc[i]+1

#half buy ct
half_buy_ct_he_vector=pd.read_csv('half_buy_ct_he_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_he_vector.iloc[i]=half_buy_ct_he_vector.iloc[i]+1

#full eco ct
full_eco_ct_he_vector=pd.read_csv('full_eco_ct_he_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_he_vector.iloc[i]=full_eco_ct_he_vector.iloc[i]+1

#full buy t
full_buy_t_he_vector=pd.read_csv('full_buy_t_he_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_he_vector.iloc[i]=full_buy_t_he_vector.iloc[i]+1

#half buy t
half_buy_t_he_vector=pd.read_csv('half_buy_t_he_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_he_vector.iloc[i]=half_buy_t_he_vector.iloc[i]+1

#full eco t
full_eco_t_he_vector=pd.read_csv('full_eco_t_he_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['throwerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['throwerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['throwerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['throwerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['throwerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['throwerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_he_vector.iloc[i]=full_eco_t_he_vector.iloc[i]+1

full_buy_ct_he_vector.to_csv('full_buy_ct_he_vector.csv',index=False)
half_buy_ct_he_vector.to_csv('half_buy_ct_he_vector.csv',index=False)
full_eco_ct_he_vector.to_csv('full_eco_ct_he_vector.csv',index=False)
full_buy_t_he_vector.to_csv('full_buy_t_he_vector.csv',index=False)
half_buy_t_he_vector.to_csv('half_buy_t_he_vector.csv',index=False)
full_eco_t_he_vector.to_csv('full_eco_t_he_vector.csv',index=False)

del full_buy_ct_he_vector,half_buy_ct_he_vector,full_eco_ct_he_vector,full_buy_t_he_vector,half_buy_t_he_vector,full_eco_t_he_vector

#flashes for flashduration

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['flashes'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['flashes'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['flashes'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['flashes'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['flashes'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['flashes'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['attackerName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_enemyflashtime_vector=pd.read_csv('full_buy_ct_enemyflashtime_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_enemyflashtime_vector.iloc[i]=full_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy ct
half_buy_ct_enemyflashtime_vector=pd.read_csv('half_buy_ct_enemyflashtime_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_enemyflashtime_vector.iloc[i]=half_buy_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco ct
full_eco_ct_enemyflashtime_vector=pd.read_csv('full_eco_ct_enemyflashtime_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_enemyflashtime_vector.iloc[i]=full_eco_ct_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full buy t
full_buy_t_enemyflashtime_vector=pd.read_csv('full_buy_t_enemyflashtime_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_enemyflashtime_vector.iloc[i]=full_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#half buy t
half_buy_t_enemyflashtime_vector=pd.read_csv('half_buy_t_enemyflashtime_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_enemyflashtime_vector.iloc[i]=half_buy_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

#full eco t
full_eco_t_enemyflashtime_vector=pd.read_csv('full_eco_t_enemyflashtime_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_enemyflashtime_vector.iloc[i]=full_eco_t_enemyflashtime_vector.iloc[i]+j['flashDuration']

full_buy_ct_enemyflashtime_vector.to_csv('full_buy_ct_enemyflashtime_vector.csv',index=False)
half_buy_ct_enemyflashtime_vector.to_csv('half_buy_ct_enemyflashtime_vector.csv',index=False)
full_eco_ct_enemyflashtime_vector.to_csv('full_eco_ct_enemyflashtime_vector.csv',index=False)
full_buy_t_enemyflashtime_vector.to_csv('full_buy_t_enemyflashtime_vector.csv',index=False)
half_buy_t_enemyflashtime_vector.to_csv('half_buy_t_enemyflashtime_vector.csv',index=False)
full_eco_t_enemyflashtime_vector.to_csv('full_eco_t_enemyflashtime_vector.csv',index=False)

del full_buy_ct_enemyflashtime_vector,half_buy_ct_enemyflashtime_vector,full_eco_ct_enemyflashtime_vector,full_buy_t_enemyflashtime_vector,half_buy_t_enemyflashtime_vector,full_eco_t_enemyflashtime_vector

#bomb events

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['bombEvents'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['bombEvents'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['bombEvents'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['bombEvents'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['bombEvents'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['bombEvents'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['playerName']==pro and i['bombAction']=='plant':
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#flash time in csv

#full buy ct
full_buy_ct_bombplant_vector=pd.read_csv('full_buy_ct_bombplant_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_bombplant_vector.iloc[i]=full_buy_ct_bombplant_vector.iloc[i]+1

#half buy ct
half_buy_ct_bombplant_vector=pd.read_csv('half_buy_ct_bombplant_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_bombplant_vector.iloc[i]=half_buy_ct_bombplant_vector.iloc[i]+1

#full eco ct
full_eco_ct_bombplant_vector=pd.read_csv('full_eco_ct_bombplant_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_bombplant_vector.iloc[i]=full_eco_ct_bombplant_vector.iloc[i]+1

#full buy t
full_buy_t_bombplant_vector=pd.read_csv('full_buy_t_bombplant_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_bombplant_vector.iloc[i]=full_buy_t_bombplant_vector.iloc[i]+1

#half buy t
half_buy_t_bombplant_vector=pd.read_csv('half_buy_t_bombplant_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_bombplant_vector.iloc[i]=half_buy_t_bombplant_vector.iloc[i]+1

#full eco t
full_eco_t_bombplant_vector=pd.read_csv('full_eco_t_bombplant_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['playerX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['playerX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['playerY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['playerY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['playerZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['playerZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_bombplant_vector.iloc[i]=full_eco_t_bombplant_vector.iloc[i]+1

full_buy_ct_bombplant_vector.to_csv('full_buy_ct_bombplant_vector.csv',index=False)
half_buy_ct_bombplant_vector.to_csv('half_buy_ct_bombplant_vector.csv',index=False)
full_eco_ct_bombplant_vector.to_csv('full_eco_ct_bombplant_vector.csv',index=False)
full_buy_t_bombplant_vector.to_csv('full_buy_t_bombplant_vector.csv',index=False)
half_buy_t_bombplant_vector.to_csv('half_buy_t_bombplant_vector.csv',index=False)
full_eco_t_bombplant_vector.to_csv('full_eco_t_bombplant_vector.csv',index=False)

del full_buy_ct_bombplant_vector,half_buy_ct_bombplant_vector,full_eco_ct_bombplant_vector,full_buy_t_bombplant_vector,half_buy_t_bombplant_vector,full_eco_t_bombplant_vector

full_buy_ct_temp=[]
temp2=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['kills'])

full_buy_ct_temp = [item for sublist in full_buy_ct_temp for item in sublist]

for i in full_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_ct_temp=temp2
del temp2

#do it for half buy ct
half_buy_ct_temp=[]
temp2=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['kills'])

half_buy_ct_temp = [item for sublist in half_buy_ct_temp for item in sublist]

for i in half_buy_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_ct_temp=temp2
del temp2

#full eco ct
full_eco_ct_temp=[]
temp2=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['kills'])

full_eco_ct_temp = [item for sublist in full_eco_ct_temp for item in sublist]

for i in full_eco_ct_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_ct_temp=temp2
del temp2



#now for t



full_buy_t_temp=[]
temp2=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['kills'])

full_buy_t_temp = [item for sublist in full_buy_t_temp for item in sublist]

for i in full_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_buy_t_temp=temp2
del temp2

#do it for half buy t
half_buy_t_temp=[]
temp2=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['kills'])

half_buy_t_temp = [item for sublist in half_buy_t_temp for item in sublist]

for i in half_buy_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

half_buy_t_temp=temp2
del temp2

#full eco t
full_eco_t_temp=[]
temp2=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['kills'])

full_eco_t_temp = [item for sublist in full_eco_t_temp for item in sublist]

for i in full_eco_t_temp:
    if i['victimName']==pro:
        temp2.append(i)

full_eco_t_temp=temp2
del temp2

#this cell does it for deaths

#full buy ct
full_buy_ct_death_vector=pd.read_csv('full_buy_ct_death_vector.csv')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_death_vector.iloc[i]=full_buy_ct_death_vector.iloc[i]+1

#half buy ct
half_buy_ct_death_vector=pd.read_csv('half_buy_ct_death_vector.csv')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_ct_death_vector.iloc[i]=half_buy_ct_death_vector.iloc[i]+1

#full eco ct
full_eco_ct_death_vector=pd.read_csv('full_eco_ct_death_vector.csv')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_death_vector.iloc[i]=full_eco_ct_death_vector.iloc[i]+1

#full buy t
full_buy_t_death_vector=pd.read_csv('full_buy_t_death_vector.csv')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_death_vector.iloc[i]=full_buy_t_death_vector.iloc[i]+1

#half buy t
half_buy_t_death_vector=pd.read_csv('half_buy_t_death_vector.csv')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_death_vector.iloc[i]=half_buy_t_death_vector.iloc[i]+1

#full eco t
full_eco_t_death_vector=pd.read_csv('full_eco_t_death_vector.csv')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j['victimX']>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j['victimX']<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j['victimY']>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j['victimY']<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j['victimZ']>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j['victimZ']<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_death_vector.iloc[i]=full_eco_t_death_vector.iloc[i]+1

full_buy_ct_death_vector.to_csv('full_buy_ct_death_vector.csv',index=False)
half_buy_ct_death_vector.to_csv('half_buy_ct_death_vector.csv',index=False)
full_eco_ct_death_vector.to_csv('full_eco_ct_death_vector.csv',index=False)
full_buy_t_death_vector.to_csv('full_buy_t_death_vector.csv',index=False)
half_buy_t_death_vector.to_csv('half_buy_t_death_vector.csv',index=False)
full_eco_t_death_vector.to_csv('full_eco_t_death_vector.csv',index=False)

del full_buy_ct_death_vector,half_buy_ct_death_vector,full_eco_ct_death_vector,full_buy_t_death_vector,half_buy_t_death_vector,full_eco_t_death_vector

#position heatmap time
full_buy_ct_temp=[]
temp=[]

for item in full_buy_ct_rounds:
    full_buy_ct_temp.append(item['frames']) #add list of frames i.e. list of dicts

    for i in full_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_buy_ct_temp=temp
del temp

#do it for half buy ct
half_buy_ct_temp=[]
temp=[]

for item in half_buy_ct_rounds:
    half_buy_ct_temp.append(item['frames'])
    
    for i in half_buy_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_ct_temp=temp
del temp

#full eco ct
full_eco_ct_temp=[]
temp=[]

for item in full_eco_ct_rounds:
    full_eco_ct_temp.append(item['frames'])
    
    for i in full_eco_ct_rounds:
        for j in i['frames']:
            for k in j['ct']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_ct_temp=temp
del temp



#now for t



full_buy_t_temp=[]
temp=[]

for item in full_buy_t_rounds:
    full_buy_t_temp.append(item['frames'])
    
    for i in full_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])


full_buy_t_temp=temp
del temp

#do it for half buy t
half_buy_t_temp=[]
temp=[]

for item in half_buy_t_rounds:
    half_buy_t_temp.append(item['frames'])

    for i in half_buy_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

half_buy_t_temp=temp
del temp

#full eco t
full_eco_t_temp=[]
temp=[]

for item in full_eco_t_rounds:
    full_eco_t_temp.append(item['frames'])
    
    for i in full_eco_t_rounds:
        for j in i['frames']:
            for k in j['t']['players']:
                if k['name']==pro:
                    temp.append([k['x'],k['y'],k['z']])

full_eco_t_temp=temp
del temp

full_buy_ct_position_vector=pd.read_csv('full_buy_ct_position_vector.csv')
full_buy_ct_mapcontrolpersecond_vector=pd.read_csv('full_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_ct_mapcontrolpersecond_vector=np.squeeze(full_buy_ct_mapcontrolpersecond_vector)
print('reached full buy ct')
for j in full_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_ct_position_vector.iloc[i]=full_buy_ct_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_ct_position_vector.to_csv('full_buy_ct_position_vector.csv',index=False)
pd.DataFrame(full_buy_ct_mapcontrolpersecond_vector).to_csv('full_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del full_buy_ct_position_vector, full_buy_ct_mapcontrolpersecond_vector

#half buy ct
half_buy_ct_position_vector=pd.read_csv('half_buy_ct_position_vector.csv')
half_buy_ct_mapcontrolpersecond_vector=pd.read_csv('half_buy_ct_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_ct_mapcontrolpersecond_vector=np.squeeze(half_buy_ct_mapcontrolpersecond_vector)
print('reached half buy ct')
for j in half_buy_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            print("found a tile in half buy ct")
            half_buy_ct_position_vector.iloc[i]=half_buy_ct_position_vector.iloc[i]+1
            half_buy_ct_mapcontrolpersecond_vector=np.add(half_buy_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_ct_position_vector.to_csv('half_buy_ct_position_vector.csv',index=False)
pd.DataFrame(half_buy_ct_mapcontrolpersecond_vector).to_csv('half_buy_ct_mapcontrolpersecond_vector.csv',index=False)
del half_buy_ct_position_vector, half_buy_ct_mapcontrolpersecond_vector

#full eco ct
full_eco_ct_position_vector=pd.read_csv('full_eco_ct_position_vector.csv')
full_eco_ct_mapcontrolpersecond_vector=pd.read_csv('full_eco_ct_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_ct_mapcontrolpersecond_vector=np.squeeze(full_eco_ct_mapcontrolpersecond_vector)
print('reached full eco ct')
for j in full_eco_ct_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_ct_position_vector.iloc[i]=full_eco_ct_position_vector.iloc[i]+1
            full_eco_ct_mapcontrolpersecond_vector=np.add(full_eco_ct_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_eco_ct_position_vector.to_csv('full_eco_ct_position_vector.csv',index=False)
pd.DataFrame(full_eco_ct_mapcontrolpersecond_vector).to_csv('full_eco_ct_mapcontrolpersecond_vector.csv',index=False)
del full_eco_ct_position_vector, full_eco_ct_mapcontrolpersecond_vector

#full buy t
full_buy_t_position_vector=pd.read_csv('full_buy_t_position_vector.csv')
full_buy_t_mapcontrolpersecond_vector=pd.read_csv('full_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
full_buy_t_mapcontrolpersecond_vector=np.squeeze(full_buy_t_mapcontrolpersecond_vector)
print('reached full buy t')
for j in full_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_buy_t_position_vector.iloc[i]=full_buy_t_position_vector.iloc[i]+1
            full_buy_ct_mapcontrolpersecond_vector=np.add(full_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

full_buy_t_position_vector.to_csv('full_buy_t_position_vector.csv',index=False)
pd.DataFrame(full_buy_t_mapcontrolpersecond_vector).to_csv('full_buy_t_mapcontrolpersecond_vector.csv',index=False)
del full_buy_t_position_vector, full_buy_t_mapcontrolpersecond_vector

#half buy t
half_buy_t_position_vector=pd.read_csv('half_buy_t_position_vector.csv')
half_buy_t_mapcontrolpersecond_vector=pd.read_csv('half_buy_t_mapcontrolpersecond_vector.csv').to_numpy()
half_buy_t_mapcontrolpersecond_vector=np.squeeze(half_buy_t_mapcontrolpersecond_vector)
print('reached half buy t')
for j in half_buy_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            half_buy_t_position_vector.iloc[i]=half_buy_t_position_vector.iloc[i]+1
            half_buy_t_mapcontrolpersecond_vector=np.add(half_buy_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))

half_buy_t_position_vector.to_csv('half_buy_t_position_vector.csv',index=False)
pd.DataFrame(half_buy_t_mapcontrolpersecond_vector).to_csv('half_buy_t_mapcontrolpersecond_vector.csv',index=False)

del half_buy_t_position_vector, half_buy_t_mapcontrolpersecond_vector

#full eco t
full_eco_t_position_vector=pd.read_csv('full_eco_t_position_vector.csv')
full_eco_t_mapcontrolpersecond_vector=pd.read_csv('full_eco_t_mapcontrolpersecond_vector.csv').to_numpy()
full_eco_t_mapcontrolpersecond_vector=np.squeeze(full_eco_t_mapcontrolpersecond_vector)
print('reached full eco t')
for j in full_eco_t_temp:
    for i in range(vectorlength):
        if (j[0]>=tiles.iloc[i,3] #to the RIGHT of WEST X
            and j[0]<=tiles.iloc[i,6] #to the LEFT of EAST X
            and j[1]>=tiles.iloc[i,4] #to the DOWN of NORTH Y
            and j[1]<=tiles.iloc[i,7] #to the UP of SOUTH Y
            and j[2]>=tiles.iloc[i,5] #to the DOWN of NORTH Z
            and j[2]<=tiles.iloc[i,8]): #to the UP of SOUTH Z
            full_eco_t_position_vector.iloc[i]=full_eco_t_position_vector.iloc[i]+1
            full_eco_t_mapcontrolpersecond_vector=np.add(full_eco_t_mapcontrolpersecond_vector,add_to_mapcontrol(tiles.iloc[i,1]))
            
full_eco_t_position_vector.to_csv('full_eco_t_position_vector.csv',index=False)
pd.DataFrame(full_eco_t_mapcontrolpersecond_vector).to_csv('full_eco_t_mapcontrolpersecond_vector.csv',index=False)
del full_eco_t_position_vector, full_eco_t_mapcontrolpersecond_vector
print('reached the end')

sourcepath='.'
sourcefiles = os.listdir(sourcepath)
destinationpath = 'Buzz/inferno'
for file in sourcefiles:
    if file.endswith('.csv'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))